In [ ]:
#default_exp Callbacks.Custom

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Fastai2 Custom Callbacks
> Creating your own Callbacks to create more dynamic networks with fastai2.

# Tests Setup
Getting dataset to setup tests.

In [ ]:
#export
from fastai2.vision.all import *
from fastai2.data.all import *
from nbdev.showdoc import *

In [ ]:
#exports
path = untar_data(URLs.PETS)
path_img, path_anno = [path/folder for folder in path.ls()]

In [ ]:
#exports
pat = r'/(\w+)_\d+.jpg'

dblock = DataBlock((ImageBlock,CategoryBlock),
                    splitter=RandomSplitter(),
                    get_items=get_image_files,
                    item_tfms=Resize(224),
                    batch_tfms=aug_transforms(),
                    get_y=RegexLabeller(pat))

In [ ]:
#exports
dls = dblock.dataloaders(path_img, bs=32)

# Custom Callbacks
Reviewing what the callbacks can do.

In [ ]:
from fastai2.callback.core import Callback

In [ ]:
show_doc(Callback)

<h2 id="Callback" class="doc_header"><code>class</code> <code>Callback</code><a href="https://github.com/fastai/fastai2/tree/master/fastai2/callback/core.py#L14" class="source_link" style="float:right">[source]</a></h2>

> <code>Callback</code>() :: `GetAttr`

Basic class handling tweaks of the training loop by changing a `Learner` in various events

In [ ]:
show_doc(Callback.__call__)

<h4 id="Callback.__call__" class="doc_header"><code>Callback.__call__</code><a href="https://github.com/fastai/fastai2/tree/master/fastai2/callback/core.py#L19" class="source_link" style="float:right">[source]</a></h4>

> <code>Callback.__call__</code>(**`event_name`**)

Call `self.{event_name}` if it's defined

In [ ]:
show_doc(Callback.__getattr__)

<h4 id="GetAttr.__getattr__" class="doc_header"><code>GetAttr.__getattr__</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/foundation.py#L226" class="source_link" style="float:right">[source]</a></h4>

> <code>GetAttr.__getattr__</code>(**`k`**)

